In [1]:
%set_env OMP_NUM_THREADS=20

env: OMP_NUM_THREADS=20


## $$\textit{import Package}$$

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
%matplotlib Qt5
from data import *
from twoBodies import TwoBody
from einops import rearrange, repeat

## $$\textit{Initialization}$$

In [3]:
np.random.seed(2)
np.set_printoptions(threshold = np.inf)

## $$\textit{Graph Setting}$$

In [4]:
fig = plt.figure(figsize=[10,4], dpi=256)
ax1, ax2 = fig.subplots(1, 2)
print(type(ax1))

<class 'matplotlib.axes._axes.Axes'>


In [5]:
nbodies    = 2
tStart     = 0.
tStep      = 200
interval   = 0.05
mass       = 1.0
min_radius = 1.0
max_radius = 1.0
maxIter    = 500
orbit_noise = 0.1

model = TwoBody(
    Nbodies    = nbodies,
    tStart     = tStart,
    tStep      = tStep,
    interval   = interval,
    mass       = mass,
    min_radius = min_radius,
    max_radius = max_radius,
    orbit_noise = orbit_noise,
)

trajs, derror, verror, traderror, traverror = model.current_loop(maxIter)
times = model.times.copy()
# print(model.times)

100%|██████████| 200/200 [00:49<00:00,  4.08it/s]


In [6]:
# state = state_generator(
#     nbodies = 2, 
#     mass = 35.0,
#     min_radius = 1.0,
#     max_radius = 3.0,
# )
# orbit, settings = get_orbit(state, t_start = 0., t_step = 500, interval = 0.05, rtol = 1e-12)

color = np.linspace(0., 1., times.shape[0])

color = repeat(color, "n -> n repeat", repeat = 3)
color[:, 0] = 0.5
color[:, 2] = color[:, 2][::-1] *0.5 + 0.25
color[:, 1] = color[:, 1] *0.5 + 0.25


orbit = trajs
invorbit = rearrange(orbit, "n m d -> d m n")
times = repeat(times, "n -> repeat n", repeat = 2)
color = repeat(color, "n m -> repeat n m", repeat = 2)


# p1: np.ndarray = orbit[..., 0, 1:3]
# p2: np.ndarray = orbit[..., 1, 1:3]

# pax1, pay1 = np.split(p1, 2, axis = -1)
# # pax1, pay1 = pax1.squeeze(axis = 1), pay1.squeeze(axis = 1)
# pax2, pay2 = np.split(p2, 2, axis = -1)
# pax2, pay2 = pax2.squeeze(axis = 1), pay2.squeeze(axis = 1)

pam1, pax1, pay1, pavx1, pavy1 = invorbit[:,0]
pam2, pax2, pay2, pavx2, pavy2 = invorbit[:,1]

m, x, y, vx, vy = invorbit
x, y, vx, vy, color, times = map(lambda x: rearrange(x, "n m ... -> (n m) ..."), (x, y, vx, vy, color, times))

print(pax1.shape, pax2.shape, pay1.shape, pay2.shape)
x1, y1, x2, y2 = [], [], [], []

x_min = min(pax1.min(), pax2.min())
x_max = max(pax1.max(), pax2.max())
y_min = min(pay1.min(), pay2.min())
y_max = max(pay1.max(), pay2.max())

# ax1.plot(pax1, pay1, 'g-', pax2, pay2, 'r-', markersize = 3)
t = ax1.scatter(x, y, s = times*2, c = color)
# cb = fig.colorbar(t, ax = ax1, orientation = 'horizontal')
# cb.set_ticks([0., 1.0])  # 设置刻度位置
# cb.set_ticklabels([model.times[0], model.times[-1]])
# cb.set_label('$\Delta t$', loc = "center")

sub1_line1, = ax1.plot(x1, y1, 'o', animated = True, markersize = 7)
sub1_line2, = ax1.plot(x2, y2, 'o', animated = True, markersize = 7)

(201,) (201,) (201,) (201,)


In [7]:
potential = potential_energy(orbit)
kinetic = kinetic_energy(orbit)
total = total_energy(orbit)
# print(total)
# np.savetxt('total_energy', total)
times = model.times
print(potential.shape, kinetic.shape, total.shape, times.shape)
pe, ke, tote, t = [], [], [], []

# pe_line, ke_line, tote_line = ax2.plot(pe, t, ke, t, tote, t, 'o', animated = True)
ax2.plot(times, potential, 'o', markersize = 3)
ax2.plot(times, kinetic, 'o', markersize = 3)
ax2.plot(times, total, 'o', markersize = 3)
pe_line, = ax2.plot(t, pe, 'o', animated = True, markersize = 7)
ke_line, = ax2.plot(t, ke, 'o', animated = True, markersize = 7)
tote_line, = ax2.plot(t, tote, 'o', animated = True, markersize = 7)
tmin, tmax = times.min(), times.max()

e_min = min(potential.min(), kinetic.min(), total.min(), 0)
e_max = max(potential.max(), kinetic.max(), total.max())

print(f'min energy: {e_min}\nmax energy: {e_max}')
print(type(pe_line), type(ke_line), type(tote_line))

print(pax1.shape, pay1.shape, pax2.shape, pay2.shape, potential.shape, kinetic.shape, total.shape, times.shape)

(201,) (201,) (201,) (201,)
min energy: -0.7898521853603532
max energy: 0.5458065555475266
<class 'matplotlib.lines.Line2D'> <class 'matplotlib.lines.Line2D'> <class 'matplotlib.lines.Line2D'>
(201,) (201,) (201,) (201,) (201,) (201,) (201,) (201,)


In [8]:
frames = np.stack([pax1, pay1, pax2, pay2, potential, kinetic, total, times], axis = 1)
timesteps = 1
frames = frames[::timesteps]
def init():
    qscale = 0.5
    escale = 0.5

    # coord initialization
    ax1.set_xlabel('$q_{x}$', rotation = 0) ; ax1.set_ylabel('$q_{y}$')
    ax1.set_axis_on()
    ax1.set_xlim(x_min - qscale * abs(x_min), x_max + qscale * abs(x_max))
    ax1.set_ylim(y_min - qscale * abs(y_min), y_max + qscale * abs(y_max))
    ax1.set_title('Trajectories')

    # energy initialization
    ax2.set_axis_on()
    ax2.set_title("Energy")
    ax2.set_xlabel('time')
    ax2.set_xlim(tmin - 1.0, tmax + 1.0)
    ax2.set_ylim(e_min - escale * abs(e_min), e_max + escale * abs(e_max))

    ax1.legend([sub1_line1, sub1_line2], ['body 1 path','body 2 path'],fontsize = 8)
    ax2.legend([pe_line, ke_line, tote_line], ['potential', 'kinetic', 'total'], fontsize = 8)

    return sub1_line1, sub1_line2, pe_line, ke_line, tote_line


def update(frame, sub1_line1, sub1_line2, pe_line, ke_line, tote_line):

    

    px1, py1, px2, py2, p, k, tt, t_interval = frame
    # print(px1, py1, px2, py2, p, k, tt, t_interval)
    
    # update coord
    # x1.append(px1);y1.append(py1)
    # x2.append(px2);y2.append(py2)
    # if len(x1) > 580:
    #     x1.pop(0);y1.pop(0);x2.pop(0);y2.pop(0)
    # sub1_line1.set_data(x1, y1)
    # sub1_line2.set_data(x2, y2)

    sub1_line1.set_data(px1, py1)
    sub1_line2.set_data(px2, py2)

    # # update energy
    # pe.append(p);ke.append(k)
    # tote.append(tt);t.append(t_interval)
    # if len(pe) > 580:
    #     pe.pop(0);ke.pop(0);tote.pop(0);t.pop(0)
    # pe_line.set_data(t, pe)
    # ke_line.set_data(t, ke)
    # tote_line.set_data(t, tote)

    pe_line.set_data(t_interval, p)
    ke_line.set_data(t_interval, k)
    tote_line.set_data(t_interval, tt)

    ax1.legend([sub1_line1, sub1_line2], ['body 1 path','body 2 path'], fontsize = 8)
    ax2.legend([pe_line, ke_line, tote_line], ['potential', 'kinetic', 'total'], fontsize = 8)
    return sub1_line1, sub1_line2, pe_line, ke_line, tote_line

## $$\textit{Graph Generation}$$

In [9]:
ani = animation.FuncAnimation(fig = fig, func = update, frames = frames, fargs = [sub1_line1, sub1_line2, pe_line, ke_line, tote_line], init_func = init, interval = 1, blit = True)
plt.tight_layout()
plt.show()

In [10]:
ani.save(f'twobody_{interval}.gif', writer = 'imagemagick', dpi = 100, progress_callback = lambda i, n: print(f'Saving frame {i}/{n}', end = '\r'))
# print(trajs)

C:\Users\caohm\AppData\Local\Temp\ipykernel_22448\2826314124.py:43: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  sub1_line1.set_data(px1, py1)
C:\Users\caohm\AppData\Local\Temp\ipykernel_22448\2826314124.py:44: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  sub1_line2.set_data(px2, py2)
C:\Users\caohm\AppData\Local\Temp\ipykernel_22448\2826314124.py:55: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  pe_line.set_data(t_interval, p)
C:\Users\caohm\AppData\Local\Temp\ipykernel_22448\2826314124.py:56: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  ke_line.set_data(t_interval, k)
C:\Users\caohm\AppData\Local\Temp\ipykernel_22448\28